In [1]:
import pandas as pd
import numpy as np
import keras
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from keras.layers import Conv2D, Dense, Activation
from functools import reduce
from keras.models import Sequential
import pickle
from sklearn.model_selection import train_test_split

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Getting directory list

In [2]:
# f = Image.open('./fish/220px-Alligator_Gar_10.JPG.jpg', "r")
d = 'fish/'
# d = 'created-data/'
dirs = [d + x.strip(' ') + '/' for x in os.listdir(d)]
temp = []

# Removing DS store
for d in dirs:
    if d == 'fish/.DS_Store':
        continue
    temp.append(d)
dirs = temp

### Reading each image in a directory


In [3]:
def get_image_dict():
    fish_data = {}

    for directory in dirs:
        try:
            dir_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
        except NotADirectoryError:
            continue

        images = []
        print("Start loading images for: " + directory)
        for image_file in dir_files:
            try:
                f = Image.open(directory + image_file)
                img_as_arr = np.array(f.resize((256,256)))
                images.append(img_as_arr)
                f.close()
            except OSError:
                continue
        fish_data[directory.split("/")[1]] = images
    return fish_data

fish_data = get_image_dict()

Start loading images for: fish/bluegill/
Start loading images for: fish/smallmouth_bass/
Start loading images for: fish/blue_catfish/
Start loading images for: fish/walleye/
Start loading images for: fish/black_crappie/
Start loading images for: fish/channel_catfish/
Start loading images for: fish/Green Sunfish/
Start loading images for: fish/alligator_gar/
Start loading images for: fish/rainbow_trout/
Start loading images for: fish/common_carp/
Start loading images for: fish/northern_pike/


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 9. 
  warnings.warn(str(msg))


Start loading images for: fish/Largemouth Bass/


In [4]:
def dict_to_vec(fish_data):
    """
    :returns: x_train, y_train
    """
    x_train = []
    y_train = []
    for key in fish_data:
        data = fish_data[key]
        
        samples = []
        for image in data:
            if len(image.shape) != 3 or image.shape[2] != 3:
                continue
            samples.append(image)

        shape = (len(samples),) + data[0].shape

        x_placeholder = np.zeros(shape)
        

        for i, sample in enumerate(samples):
            x_placeholder[i] = sample
        
        x_train.append(x_placeholder)
        y_train.append(key)
        
    return x_train, y_train


In [5]:
x_train_vec, labels = dict_to_vec(fish_data)

In [6]:
# zipped_list = list(zip(x_train_vec, labels))

In [7]:
# , [labels[0]]
# list(zip(x_train_vec, labels))[0]

In [8]:
# with open('training_data.pkl', 'wb') as f:
#     pickle.dump(zippped_list, f)
#     f.close()

# with open('training_data.pkl', 'r') as f:
#     pickle.load(f)

In [9]:
# Removing walleye
# if x_train_vec[3].shape[0] < 1000:
#     x_train_vec.pop(3)

In [10]:
[x.shape for x in x_train_vec]

[(50, 256, 256, 3),
 (49, 256, 256, 3),
 (68, 256, 256, 3),
 (32, 256, 256, 3),
 (52, 256, 256, 3),
 (52, 256, 256, 3),
 (56, 256, 256, 3),
 (60, 256, 256, 3),
 (41, 256, 256, 3),
 (42, 256, 256, 3),
 (67, 256, 256, 3),
 (42, 256, 256, 3)]

In [22]:
list_num_index = []
for vec in x_train_vec:
    list_num_index.append(vec.shape[0])

In [23]:
# for i in range(len(x_train_vec)):
#     x_train_vec[i] = x_train_vec[i][0:1000,:,:,:]

In [25]:
sum(list_num_index)

611

In [33]:
y = []

for i, num_examples in enumerate(list_num_index):
    y.append([i] * num_examples)
# flatten

y_train = np.array([item for sublist in y for item in sublist])

In [34]:
x_train = np.concatenate(x_train_vec)

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [40]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(519, 256, 256, 3)
(92, 256, 256, 3)
(519,)
(92,)


In [42]:
# np.save('keras_model/x_train.npy', x_train)
# np.save('keras_model/y_train.npy', y)

with open('keras_model/train100.pkl', 'wb') as handle:
    pickle.dump((X_train, Y_train), handle)
    
with open('keras_model/test100.pkl', 'wb') as handle:
    pickle.dump((X_test, Y_test), handle)

In [ ]:
# with open('keras_model/train2.pkl', 'wb') as handle:
#     pickle.dump((x_train[5000:], y[5000:]), handle, protocol=4)

In [ ]:
# with open('keras_model/test.pkl', 'wb') as handle:
#     pickle.dump((X_test, y_test), handle)

In [43]:
# pickle.dump(x_train, 'x_train')
print(x_train.shape)
print(y_train.shape)

(611, 256, 256, 3)
(611,)


In [ ]:
parameters = {
    'channels': 3,
    'window': 5,
    'stride': 3,
    'padding': 'same',
    'activation': 'relu'
}

channels = parameters['channels']
window = parameters['window']
stride = parameters['stride']
padding = parameters['padding']
activation = parameters['activation']

model = Sequential()

# magic numbers: 256 * 256 * 3 is the total pixels per image
model.add(Dense((256 * 256 * 3), input_shape=x_train.shape))

model.add(Conv2D(filters=channels, kernel_size=window, strides=stride, padding=padding, activation=activation,
    data_format='channels_last'))
